In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import demoji
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.corpus import opinion_lexicon
from nltk.tokenize import TweetTokenizer
from wordcloud import WordCloud

In [2]:
nltk.download('stopwords')
nltk.download('opinion_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Konrad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\Konrad\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

In [3]:
# Choosing a theme for collected data
theme = 'WorldCup'

In [4]:
# Loading collected data
tweets = pd.read_csv(f'./data/{theme}/tweets.csv')
users = pd.read_csv(f'./data/{theme}/users.csv')

In [5]:
# Dropping duplicated data
tweets = tweets.drop_duplicates(subset=['id'])
users = users.drop_duplicates(subset=['id'])

In [6]:
# Getting tweets in english
tweets_en = tweets[tweets['lang'] == 'en']

In [7]:
# Preprocessing function
punctuation = ['.', ',', '|', '-', '?', '!', ':', '\'', '\"', '$', '&', '<', '>']

def preprocess(text):
    # Removing links and mentions
    text_links = re.sub(r'(@|https?)\S+', '', text)

    # Cleaning whitespaces
    text_white = re.sub(r"\s+", " ", text_links, flags=re.UNICODE)

    # Removing numbers
    text_number = re.sub(r"\d+", "", text_white)

    # Lowering text
    text_lower = text_number.lower()

    # Removing punctuation
    text_punctuation = "".join([i for i in text_lower if i not in punctuation])

    # Removing emojis
    text_emoji = demoji.replace(text_punctuation, '')

    return text_emoji

In [8]:
# Tokenizing function
tokenizer = TweetTokenizer()

def tokenize(text):
    text_words = [word for word in tokenizer.tokenize(text)]
    return text_words

In [9]:
# Stopwords removal function
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    text_stopwords = [word for word in text if word not in stop_words]

    return text_stopwords

In [10]:
# Applying preprocessing function
tweets_en['text_clean'] = tweets_en.text.apply(lambda row : preprocess(row))

C:\Users\Konrad\AppData\Local\Temp\ipykernel_19184\2831565511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_en['text_clean'] = tweets_en.text.apply(lambda row : preprocess(row))


In [11]:
# Applying tokenizing function
tweets_en['text_clean_list'] = tweets_en.text_clean.apply(lambda row : tokenize(row))

C:\Users\Konrad\AppData\Local\Temp\ipykernel_19184\2381255106.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_en['text_clean_list'] = tweets_en.text_clean.apply(lambda row : tokenize(row))


In [12]:
# Applying stopwords removal function
tweets_en['text_clean_stopwords'] = tweets_en.text_clean_list.apply(lambda row : remove_stopwords(row))

C:\Users\Konrad\AppData\Local\Temp\ipykernel_19184\4086829583.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_en['text_clean_stopwords'] = tweets_en.text_clean_list.apply(lambda row : remove_stopwords(row))


In [13]:
# Obtaining list of hashtags from a tweet
# tweets_en['hashtags_list'] = tweets_en.text_clean_list.apply(lambda row : [hashtag for hashtag in row if hashtag.startswith('#')])
tweets_en['hashtags_list'] = tweets_en.text.apply(lambda row : re.findall('#(\w+)', row))

C:\Users\Konrad\AppData\Local\Temp\ipykernel_19184\1668657259.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_en['hashtags_list'] = tweets_en.text.apply(lambda row : re.findall('#(\w+)', row))


In [14]:
# Displaying preprocessed data
tweets_en

,hashtag,id,author_id,text,like_count,reply_count,retweet_count,quote_count,created_at,lang,mentions,text_clean,text_clean_list,text_clean_stopwords,hashtags_list
2,#Qatar2022,1592601191761838081,1591065849531604994,Massage and sex in Qatar❤️\nMassage body to bo...,0,0,0,0,2022-11-15 19:31:27+00:00,en,NaN,massage and sex in qatar massage body to body ...,"[massage, and, sex, in, qatar, massage, body, ...","[massage, sex, qatar, massage, body, body, #qa...","[Qatar, Qatar2022]"
5,#Qatar2022,1592601156412649477,968203797049769984,How to identify a whitexican in Qatar 2022 #Q...,0,1,0,0,2022-11-15 19:31:19+00:00,en,NaN,how to identify a whitexican in qatar #qatar ...,"[how, to, identify, a, whitexican, in, qatar, ...","[identify, whitexican, qatar, #qatar, #mexico,...","[Qatar2022, Mexico, WorldCup2022]"
7,#Qatar2022,1592601145545031681,1585336194010955781,Gillingham vs AFC Fylde\n\n⏰ 4:45:00 PM\n\nWho...,0,0,0,0,2022-11-15 19:31:16+00:00,en,NaN,gillingham vs afc fylde pm whos going to win...,"[gillingham, vs, afc, fylde, pm, whos, going, ...","[gillingham, vs, afc, fylde, pm, whos, going, ...","[FIFAWorldCup, Qatar2022]"
8,#Qatar2022,1592601145503059968,1585336194010955781,Derby vs Torquay\n\n⏰ 4:45:00 PM\n\nWho's goin...,0,0,0,0,2022-11-15 19:31:16+00:00,en,NaN,derby vs torquay pm whos going to win #fifaw...,"[derby, vs, torquay, pm, whos, going, to, win,...","[derby, vs, torquay, pm, whos, going, win, #fi...","[FIFAWorldCup, Qatar2022]"
9,#Qatar2022,1592601145415008256,1585336194010955781,Cambridge United vs Curzon Ashton\n\n⏰ 4:45:00...,0,0,0,0,2022-11-15 19:31:16+00:00,en,NaN,cambridge united vs curzon ashton pm whos go...,"[cambridge, united, vs, curzon, ashton, pm, wh...","[cambridge, united, vs, curzon, ashton, pm, wh...","[FIFAWorldCup, Qatar2022]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461392,#Qatar2022,1605037565999980545,1499701605079396353,The coronation of the true living #GOAT! @WeAr...,0,0,0,0,2022-12-20 03:09:10+00:00,en,"['2849963640', '1058376110']",the coronation of the true living #goat #dext...,"[the, coronation, of, the, true, living, #goat...","[coronation, true, living, #goat, #dextools, e...","[GOAT, Dextools, WorldCup2022, WorldCup, Qatar..."
461393,#Qatar2022,1605037409074573313,897741929387106305,Peter Drury on Argentina’s winning moment.\n\n...,3,0,0,0,2022-12-20 03:08:33+00:00,en,NaN,peter drury on argentina’s winning moment #mes...,"[peter, drury, on, argentina, ’, s, winning, m...","[peter, drury, argentina, ’, winning, moment, ...","[Messi𓃵, ArgentinaVsFrance, Qatar2022, FIFAWor..."
461405,#Qatar2022,1605037015233310720,1363653333479788549,Argentine dairy industry #DitchDairy @dairy_tr...,0,0,0,0,2022-12-20 03:06:59+00:00,en,"['1370074650038910983', '1246189682150367233']",argentine dairy industry #ditchdairy #worldcup...,"[argentine, dairy, industry, #ditchdairy, #wor...","[argentine, dairy, industry, #ditchdairy, #wor...","[DitchDairy, WorldCupFinal, FIFAWorldCup, Qata..."
461406,#Qatar2022,1605036970660462602,1522221998151061509,This time @TeamMessi Argentina ⚽❤✌ \n\n 📕 #Dex...,0,0,0,0,2022-12-20 03:06:48+00:00,en,['1058376110'],this time argentina #dextools quack wool chz...,"[this, time, argentina, #dextools, quack, wool...","[time, argentina, #dextools, quack, wool, chz,...","[Dextools, WorldCup2022, WorldCup, Qatar2022, ..."


In [15]:
# Saving processed tweets and users datasets
tweets_en.to_csv(f'./data/{theme}/tweets_clean.csv')
users.to_csv(f'./data/{theme}/users_clean.csv')